<a href="https://colab.research.google.com/github/Luis-Alatrista/TelecomX_parte2/blob/main/Copia_de_TelecomX_parte2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Telecom X — Modelado Predictivo de Churn (Parte 2)

**Objetivo:** Construir un pipeline robusto para predecir cancelación de clientes (*churn*), usando el dataset **limpio** de la Parte 1 (`telecomx_churn_clean.csv`).

**Contenido:**
1. Preparación de datos (carga, selección de columnas relevantes, codificación, escalado opcional).
2. Análisis de desbalance de clases y opciones de balanceo (SMOTE / undersampling).
3. Visualizaciones: correlaciones, boxplots/scatter de variables clave vs churn.
4. Modelado con **2 enfoques**:
   - **Regresión Logística** (requiere normalización).
   - **Random Forest** (no requiere normalización).
5. Evaluación (accuracy, precision, recall, F1, matriz de confusión) y comparación crítica.
6. Interpretabilidad (coeficientes / importancias) e **informe final** con insights y recomendaciones.


## 1) Carga y preparación de datos

- Se utiliza el archivo limpio `telecomx_churn_clean.csv` (generado en la Parte 1).
- Se eliminan columnas que **no aportan valor predictivo** (IDs u otros identificadores únicos).
- Se detecta de forma **robusta** la columna *target* (`Churn`) y columnas relevantes por patrones.
- Se realiza **one-hot encoding** para variables categóricas.


In [ ]:

import pandas as pd
import numpy as np
import re

CSV_PATH = "/mnt/data/telecomx_churn_clean.csv"
df = pd.read_csv(CSV_PATH)

print("Shape original:", df.shape)

def find_col(patterns, cols):
    pat = re.compile("|".join(patterns), re.IGNORECASE)
    matches = [c for c in cols if pat.search(c)]
    return matches[0] if matches else None

target_col = find_col([r"^churn$", r"evas", r"cancel", r"baja"], df.columns)
if not target_col:
    raise ValueError("No se encontró columna objetivo (Churn). Ajusta patrones.")

df[target_col] = (df[target_col].astype(str).str.strip().str.lower()
                  .map({"yes":1,"y":1,"true":1,"1":1,"si":1,"sí":1,"no":0,"n":0,"false":0,"0":0})
                  .fillna(0).astype(int))

id_like = [c for c in df.columns if re.search(r"(?:^|_)id$|customerid|clientid|folio|uuid|hash", c, re.IGNORECASE)]
df = df.drop(columns=id_like, errors="ignore")
print("Columnas eliminadas por ID/no predictivas:", id_like)

numeric_cols = [c for c in df.select_dtypes(include=[np.number]).columns if c != target_col]
categorical_cols = [c for c in df.columns if c not in numeric_cols + [target_col]]

print("Numéricas:", len(numeric_cols), " | Categóricas:", len(categorical_cols))

df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

print("Shape tras OHE:", df_encoded.shape)
df_encoded.head()


## 2) Desbalance de clases

- Proporción de cancelaciones vs activos.
- Si hay desbalance, se intenta **SMOTE** como opción de balanceo (si está disponible).


In [ ]:

y = df_encoded[target_col]
X = df_encoded.drop(columns=[target_col])

ratio = y.mean()
print(f"Tasa de churn (1): {ratio:.4f}  |  Activos (0): {1-ratio:.4f}")

from collections import Counter
print("Distribución de clases:", Counter(y))

is_imbalanced = (ratio < 0.35) or (ratio > 0.65)
print("¿Desbalance notable?:", is_imbalanced)

X_bal, y_bal = X.copy(), y.copy()

if is_imbalanced:
    try:
        from imblearn.over_sampling import SMOTE
        sm = SMOTE(random_state=42)
        X_bal, y_bal = sm.fit_resample(X, y)
        print("SMOTE aplicado. Nueva distribución:", Counter(y_bal))
    except Exception as e:
        print("No se pudo aplicar SMOTE. Continuamos sin oversampling. Detalle:", e)


## 3) Visualizaciones (correlaciones y relaciones con churn)

- **Matriz de correlación** (numéricas)
- **Tiempo de contrato (tenure) × Churn** (boxplot)
- **Gasto total (TotalCharges) × Churn** (boxplot y scatter)


In [ ]:

import matplotlib.pyplot as plt

num_cols_for_corr = [c for c in X.columns if X[c].dtype != 'uint8']
num_subset = df[num_cols_for_corr + [target_col]].select_dtypes(include=[np.number])
corr = num_subset.corr()

plt.figure()
plt.imshow(corr, interpolation='nearest')
plt.title('Matriz de Correlación (numéricas)')
plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
plt.yticks(range(len(corr.index)), corr.index)
plt.colorbar()
plt.tight_layout()
plt.show()


In [ ]:

import re, matplotlib.pyplot as plt

tenure_col = None
for c in df.columns:
    if re.search(r"tenure|antig", c, re.IGNORECASE):
        tenure_col = c
        break

if tenure_col:
    plt.figure()
    groups = [df[df[target_col]==g][tenure_col].dropna() for g in sorted(df[target_col].unique())]
    if all(len(g)>0 for g in groups):
        plt.boxplot(groups, labels=[str(int(v)) for v in sorted(df[target_col].unique())])
        plt.title(f'Tenure por Churn (0=No,1=Sí) — {tenure_col}')
        plt.ylabel(tenure_col)
        plt.tight_layout()
        plt.show()
else:
    print("No se encontró columna de Tenure.")


In [ ]:

import re, matplotlib.pyplot as plt

total_col = None
for c in df.columns:
    if re.search(r"total.*charge|cargos?.*total", c, re.IGNORECASE):
        total_col = c
        break

if total_col:
    plt.figure()
    groups = [df[df[target_col]==g][total_col].dropna() for g in sorted(df[target_col].unique())]
    if all(len(g)>0 for g in groups):
        plt.boxplot(groups, labels=[str(int(v)) for v in sorted(df[target_col].unique())])
        plt.title(f'{total_col} por Churn (0=No,1=Sí)')
        plt.ylabel(total_col)
    plt.tight_layout()
    plt.show()

    sample = df.sample(min(3000, len(df)), random_state=42)
    plt.figure()
    plt.scatter(sample[total_col], sample[target_col], alpha=0.3)
    plt.title(f'Scatter: {total_col} vs Churn (0/1)')
    plt.xlabel(total_col)
    plt.ylabel('Churn')
    plt.tight_layout()
    plt.show()
else:
    print("No se encontró columna de TotalCharges.")


## 4) División Train/Test (80/20)

In [ ]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_bal, y_bal, test_size=0.2, random_state=42, stratify=y_bal)
X_train.shape, X_test.shape, y_train.mean(), y_test.mean()


## 5) Modelos — Regresión Logística (con normalización) y Random Forest (sin normalización)

In [ ]:

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

pipe_lr = Pipeline([
    ('scaler', StandardScaler(with_mean=False)),
    ('clf', LogisticRegression(max_iter=1000))
])
pipe_lr.fit(X_train, y_train)
print("Entrenado: Regresión Logística")


In [ ]:

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=300,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)
print("Entrenado: Random Forest")


## 6) Evaluación: Accuracy, Precision, Recall, F1, Matriz de confusión

In [ ]:

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import numpy as np

def evaluate(model, X_tr, y_tr, X_te, y_te, name="Modelo"):
    y_pred_tr = model.predict(X_tr)
    y_pred_te = model.predict(X_te)

    metrics = {
        'accuracy_train': accuracy_score(y_tr, y_pred_tr),
        'precision_train': precision_score(y_tr, y_pred_tr, zero_division=0),
        'recall_train': recall_score(y_tr, y_pred_tr, zero_division=0),
        'f1_train': f1_score(y_tr, y_pred_tr, zero_division=0),
        'accuracy_test': accuracy_score(y_te, y_pred_te),
        'precision_test': precision_score(y_te, y_pred_te, zero_division=0),
        'recall_test': recall_score(y_te, y_pred_te, zero_division=0),
        'f1_test': f1_score(y_te, y_pred_te, zero_division=0),
    }

    print(f"\n{name} — Métricas:")
    for k, v in metrics.items():
        print(f"{k}: {v:.4f}")

    cm = confusion_matrix(y_te, y_pred_te)
    plt.figure()
    plt.imshow(cm, interpolation='nearest')
    plt.title(f'Matriz de Confusión — {name} (Test)')
    plt.colorbar()
    tick_marks = np.arange(2)
    plt.xticks(tick_marks, ['No', 'Sí'])
    plt.yticks(tick_marks, ['No', 'Sí'])
    plt.ylabel('Real')
    plt.xlabel('Predicción')
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, cm[i, j], ha="center", va="center")
    plt.tight_layout()
    plt.show()

    print("\nReporte de Clasificación (Test):")
    print(classification_report(y_te, y_pred_te, target_names=['No','Sí'], zero_division=0))
    return metrics

metrics_lr = evaluate(pipe_lr, X_train, y_train, X_test, y_test, name="Regresión Logística")
metrics_rf = evaluate(rf, X_train, y_train, X_test, y_test, name="Random Forest")


## 7) Interpretabilidad — Coeficientes (LR) e Importancia (RF)

In [ ]:

import pandas as pd
coef = pd.Series(pipe_lr.named_steps['clf'].coef_[0], index=X_train.columns)
coef.sort_values(ascending=False).head(15)


In [ ]:

coef.sort_values().head(15)


In [ ]:

import pandas as pd
imp = pd.Series(rf.feature_importances_, index=X_train.columns).sort_values(ascending=False)
imp.head(20)


## 8) Informe y Recomendaciones

- Comparar resultados de LR vs RF (overfitting/underfitting revisando diferencias train/test).
- Variables más influyentes según coeficientes (LR) e importancias (RF).
- Sugerencias de retención: contratos a 1–2 años, incentivar pagos automáticos, bundles de valor, alertas tempranas con umbral optimizado según objetivo (recall o precision).
